# RNN과 CNN 사용 시퀀스 처리하기
* RNN(순환신경망), 고정 길이 입력이 아닌 임의 길이를 가진 시퀀스를 다룰 수 있다.
* 문장,문서,오딩 샘플로 자동번역,자연어처리 가능(NLP)
* RNN의 두가지 난제
  * 불안정한 그레디언트는 순환 드롭아웃과 순환 층 정규화를 포함한 여러 기법으로 감소 가능
  * 제한적인 단기 기억은 LSTM과 GRU셀을 사용해 확장 가능.
* RNN이 순차 데이터를 다루는 유일한 신경망은 아니다
  * 작은 시퀀스의 경우 일반적인 밀집 네트워크가 처리할 수 있고 오디오,텍스트 처럼 긴 시퀀스는 합성곱 신경망도 실제로 잘 작동할 수 있다.
  * 마지막에 수만 개의 타임 스텝을 가진 시퀀스를 다루는 CNN층인 WaveNet을 구현하겠다.
### 순환 뉴런과 순환 층
* 순환 신경망은 뒤쪽으로 순환하는 연결도 있다는 점이 다르다.
* 각 순환 뉴런은 두 벌의 가중치를 가진다.
  * 하나는 입력 x이고 하나는 이전 스텝의 출력을 y(t-1)을 위한 것이다.
* y(t)= 현제 층의 입력과 가중치 + 이전층의 출력과 이전층의 출력 가중치.
---
### 메모리 셀
* 타임 스텝 t에서 순환 뉴런의 출력은 이전 타임 스텝의 모든 입력에 대한 함수이므로 이를 일종의 메모리 형태라고 할 수 있다.
* 타임 스텝에 걸쳐서 어떤 상태를 보존하는 신경망의 궝 요소를 메모리 셀이라 한다.
---
### 입력과 출력 시퀀스
* RNN은 입력 시퀀스를 받아 출력 시퀀스를 만들 수 있다.(시퀀스 투 시퀀스 네트워크)
  * 주식 가격 예측
* 또는 시퀀스를 주입하고 마지막을 제외한 모든 출력을 무시 할 수 있다.( 시퀀스 투 벡터 네트워크)
  * 감성점수 -1 or 1
* 하나의 입력 벡터를 반복해서 주입하고,  하나의 시퀀스를 출력할 수 있다( 벡터-투-시퀀스 네트워크)
  * 이미지를 입력해서 이미지에 대한 캡션 출력
* 인코더라 부르는 시퀀스 투 벡터 네트워크 뒤에 디코더라 부르는 시퀀스 투 시퀀스 네트워크를 연결할 수 있다.
  * 한 언어의 문장을 네트워크에 주입하면 이 문장을 인코더에서 벡터로 변환하고 디코더에서 다른 언어로 디코딩한다.
---
### RNN 훈련하기
* 타임 스텝으로 네트워크를 펼치고 보통의 역전파를 사용하는 것. 이를 BPTT라고 합니다.
* 정방향 패스가 펼쳐진 네트워크를 통과한다. 그러면 비용함수를 사용하여 출력 시퀀스가 평가된다.
* 이 비용함수는 일부 출력을 무시할 수 잇다.
* 결국 모델 파라미터는 BPTT동안 계산된 그레디언트를 사용하여 업데이트 된다.
* 그레디언트가 마지막 출력 뿐 아니라 비용함수를 사용한 모든 출력에서 역방향으로 전파된다.
* 각 타임 스텝마다 같은 매개변수 W와 b가 사용되기에 모든 타임 스텝에 걸쳐 합산 될 것이다.
---
### 시계열 예측하기
* 시간당 사용자수,온도,지표를 사용한 연구하면 이를 시계열이라 부른다.
  * 타임스텝마다 하나 이상의 값을 가진 시퀀스
* 단변량 시계열, 다변량 시계열이 있다.
* 비어있는 값을 예측하는 방법도 있다. 이를 값 대체(imputation)라고 한다.


In [1]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt

* batch_size만큼 n_steps 길이의 여러 시계열을 만든다.
* 각 시계열에는 타임 스텝마다 하나의 값만 있다. 즉 단변량
* [배치크기,타임스텝수,1]크기의 넘파이 배열을 반환한다.

In [2]:
def generate_time_series(batch_size,n_steps):
  freq1,freq2,offsets1,offsets2=np.random.rand(4,batch_size,1)
  time=np.linspace(0,1,n_steps)
  series=0.5*np.sin((time-offsets1)*(freq1*10+10))
  series += 0.2*np.sin((time))
  series +=0.1*(np.random.rand(batch_size,n_steps)-0.5)
  return series[...,np.newaxis].astype(np.float32)

In [3]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

### 기준 성능
* 기준성능을 준비하는 것이 좋다.
* 가장 간단한 방법은 시계열의 마지막 값을 그대로 예측하는 것이다. 이를 순진한 예측이라 한다. 이를 넘는 것이 매우 어렵다!

In [8]:
y_pred=X_valid[:,-1]
np.mean(keras.losses.mean_squared_error(y_valid,y_pred))

0.013103405

In [12]:
model=keras.models.Sequential([
                               keras.layers.Flatten(input_shape=[50,1]),
                               keras.layers.Dense(1)
                               ])

In [17]:
model.compile(loss="mse",optimizer="adam")
history=model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

Epoch 1/20
219/219 [==============================] - 3s 9ms/step - loss: 0.3452 - val_loss: 0.3484
Epoch 2/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3407 - val_loss: 0.3453
Epoch 3/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3446 - val_loss: 0.3423
Epoch 4/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3338 - val_loss: 0.3600
Epoch 5/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3354 - val_loss: 0.3334
Epoch 6/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3292 - val_loss: 0.3394
Epoch 7/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3254 - val_loss: 0.3315
Epoch 8/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3277 - val_loss: 0.3324
Epoch 9/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3206 - val_loss: 0.3198
Epoch 10/20
219/219 [==============================] - 2s 8ms/step - loss: 0.3179 - val_loss: 0.3186

### 간단한 RNN 구현하기
* 간단한 RNN층이다. 기본 하이퍼볼릭 탄젠ㄴ트를 사용한다.
* 입력 시퀀스의 길이를 지정할 필요가 없다. 따라서 None으로 지정함.
* 하이퍼볼릭 탄젠트를 적용해서 y0을 출력한다,
  * 기본 RNN에서는 이 출력이 새로운 상태 h0이 된다.
  * 이는 다음 입력값 과 함께 동일한 순환 뉴런으로 전달된다.
  * 이 과정이 마지막 타임 스텝까지 반복되고 마지막 값 y49를 출력한다.
  *


In [15]:
model=keras.models.Sequential([
                               keras.layers.SimpleRNN(1,input_shape=[None,1])
])

In [16]:
model.compile(loss="mse",optimizer="adam")
model.fit(X_train,y_train,epochs=10,validation_data=(X_valid,y_valid))

Epoch 1/10
219/219 [==============================] - 3s 9ms/step - loss: 1.1083 - val_loss: 1.0161
Epoch 2/10
219/219 [==============================] - 2s 8ms/step - loss: 0.8305 - val_loss: 0.5955
Epoch 3/10
219/219 [==============================] - 2s 8ms/step - loss: 0.5574 - val_loss: 0.5493
Epoch 4/10
219/219 [==============================] - 2s 8ms/step - loss: 0.4881 - val_loss: 0.4192
Epoch 5/10
219/219 [==============================] - 2s 8ms/step - loss: 0.3883 - val_loss: 0.3905
Epoch 6/10
219/219 [==============================] - 2s 8ms/step - loss: 0.3780 - val_loss: 0.3795
Epoch 7/10
219/219 [==============================] - 2s 8ms/step - loss: 0.3712 - val_loss: 0.3730
Epoch 8/10
219/219 [==============================] - 2s 8ms/step - loss: 0.3636 - val_loss: 0.3662
Epoch 9/10
219/219 [==============================] - 2s 8ms/step - loss: 0.3576 - val_loss: 0.3644
Epoch 10/10
219/219 [==============================] - 2s 8ms/step - loss: 0.3522 - val_loss: 0.3547

* 트렌드와 계절성
  * 가중 이동 평균,자동 회귀 누적 이동 평균 같이 시계열을 예측하는 다른 방법이 많다.
  * 일부는 트렌드나 계절성을 제거해야한다.
  * 선크림 판매량을 예측하려면 강한 계절성이 있는데 이는 매년 반복되므로 스텝의 값과 작년도 값의 차이를 계산하여 사용한다.
  * 훈련 이후에 계절 패턴을 최종 예측에 더한다.
  * RNN은 일반적으로 이런 작업이 모두 필요없다.
---
## 심층 RNN
* 셀을 여러층으로 쌓는 것이 일반적이다. 이를 심층 RNN이라고 한다.
* SimpleRNN층을 사용한다.

In [18]:
model=keras.models.Sequential([
                               keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                               keras.layers.SimpleRNN(20,return_sequences=True),
                               keras.layers.SimpleRNN(1)
])

In [19]:
model.compile(loss="mse",optimizer="adam")
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 8s 29ms/step - loss: 0.0324 - val_loss: 0.0039
Epoch 2/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0042 - val_loss: 0.0029
Epoch 3/20
219/219 [==============================] - 6s 28ms/step - loss: 0.0028 - val_loss: 0.0030
Epoch 4/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0026 - val_loss: 0.0021
Epoch 5/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 6/20
219/219 [==============================] - 6s 28ms/step - loss: 0.0021 - val_loss: 0.0022
Epoch 7/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0021 - val_loss: 0.0026
Epoch 8/20
219/219 [==============================] - 6s 28ms/step - loss: 0.0020 - val_loss: 0.0017
Epoch 9/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0019 - val_loss: 0.0016
Epoch 10/20
219/219 [==============================] - 6s 28ms/step - loss: 0.0019 - val_lo

* 마지막 층은 이상적이지 않다. 단변량 식{열을 예측하기 때문에 하나의 유닛이 필요하다. 이는 타임 스텝마다 하나의 출력을 만들어야 한다는 뜻이다.
* 마지막 층의 은닉상태는 크게 필요하지 않다. 출력층을 Dense층으로 바꾸는 경우가 많다. 마지막 rnn층의 return_sequences=True를 제거한다

In [21]:
model=keras.models.Sequential([
                               keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                               keras.layers.SimpleRNN(20),
                               keras.layers.Dense(1)
])

In [22]:
model.compile(loss="mse",optimizer="adam")
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid),batch_size=10)

Epoch 1/5
700/700 [==============================] - 13s 17ms/step - loss: 0.0066 - val_loss: 0.0022
Epoch 2/5
700/700 [==============================] - 11s 16ms/step - loss: 0.0021 - val_loss: 0.0020
Epoch 3/5
700/700 [==============================] - 12s 17ms/step - loss: 0.0020 - val_loss: 0.0019
Epoch 4/5
700/700 [==============================] - 12s 17ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 5/5
700/700 [==============================] - 12s 16ms/step - loss: 0.0018 - val_loss: 0.0017


#### 여러 타임 스텝 앞을 예측하기
* 다음 값 10개를 예측하려하면 이미 훈련된 모델을 사용하여 다음 값을 예측한 다음 그것을 입력으로 추가하는 것이다.

In [23]:
series=generate_time_series(1,n_steps+10)
X_new,y_new=series[:,:n_steps],series[:,n_steps:]
X=X_new
for step_ahead in range(10):
  y_pred_one=model.predict(X[:,step_ahead:])[:,np.newaxis,:]
  X=np.concatenate([X,y_pred_one],axis=1)
y_pred=X[:,n_steps:]

In [24]:
y_pred

array([[[0.2142808 ],
        [0.33213365],
        [0.4318902 ],
        [0.52033585],
        [0.5880155 ],
        [0.619589  ],
        [0.61769336],
        [0.5807548 ],
        [0.5291406 ],
        [0.46240434]]], dtype=float32)

* 두번째 방법은 RNN을 훈련하여 다음 값 10개를 한번에 예ㅖ측하는 것이다.
* 시퀀스 to 벡터 모델을 사용하지만 값 10개를 출력한다.

In [25]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [28]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=5,
                    validation_data=(X_valid, Y_valid))

Epoch 1/5
219/219 [==============================] - 6s 20ms/step - loss: 0.0482 - val_loss: 0.0105
Epoch 2/5
219/219 [==============================] - 4s 19ms/step - loss: 0.0072 - val_loss: 0.0056
Epoch 3/5
219/219 [==============================] - 4s 19ms/step - loss: 0.0047 - val_loss: 0.0036
Epoch 4/5
219/219 [==============================] - 4s 19ms/step - loss: 0.0035 - val_loss: 0.0027
Epoch 5/5
219/219 [==============================] - 4s 19ms/step - loss: 0.0033 - val_loss: 0.0028


In [32]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, -10:, :]
Y_pred = model.predict(X_new)[..., np.newaxis]
Y_pred

array([[[ 0.6741889 ],
        [ 0.61107606],
        [ 0.5625881 ],
        [ 0.5381058 ],
        [ 0.47292084],
        [ 0.40881014],
        [ 0.32036072],
        [ 0.20040798],
        [ 0.06797732],
        [-0.00368373]]], dtype=float32)

* 마지막 타임 스텝에서만 10개를 예측하도록 훈련하는 대신 모든 타임 스텝에서 10개를 예측하도록 모델을 훈련할 수 있다.
* 모든 출력에서의 출력의 항이 손실에 포함된다는 장점을 가짐.
* 시퀀스 투 시퀀스 모델로 바꾸려면 모든 순환층에 return_sequences=True 지정.
* 이런 목적을 위해 TimeDistributed 층을 제공한다. 이는 다른 층을 감싸서 시퀀스의 모든 타임 스테에 적용한다.


In [34]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [35]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred): # 마지막 타임 스텝의 출력만으로 평가를 한다.
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=5,
                    validation_data=(X_valid, Y_valid))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5
219/219 [==============================] - 6s 22ms/step - loss: 0.0291 - last_time_step_mse: 0.0269 - val_loss: 0.0148 - val_last_time_step_mse: 0.0090
Epoch 2/5
219/219 [==============================] - 4s 20ms/step - loss: 0.0144 - last_time_step_mse: 0.0087 - val_loss: 0.0137 - val_last_time_step_mse: 0.0078
Epoch 3/5
219/219 [==============================] - 4s 20ms/step - loss: 0.0129 - last_time_step_mse: 0.0072 - val_loss: 0.0118 - val_last_time_step_mse: 0.0061
Epoch 4/5
219/219 [==============================] - 4s 20ms/step - loss: 0.0117 - last_time_step_mse: 0.0061 - val_loss: 0.0117 - val_last_time_step_mse: 0.0049
Epoch 5/5
219/219 [==============================] - 4s 20ms/step - loss: 0.0112 - last_time_step_mse: 0.0064 - val_loss: 0.0106 - val_last_time_step_mse: 0.0073


* 시계열을 예측할 때 오차막대를 사용하는 것이 유용하다. 11장에서 소개한 MC드롭아웃이 좋다.
---
## 긴 시퀀스 다루기
* 많은 타임 스텝에 걸쳐 실행해야 하므로 RNN이 매우 깊은 네트워크가 된다.
* 그레디언트 소실과 폭주 문제가 생길 수 잇다.
---
### 불안정한 그레디언트 문제와 싸우기
* 해결하기 위해 심층 신경망의 많은 기법을 사용할 수 있다.
  * 좋은 가중치 초기화, 빠른 옵티마이저, 드롭아웃 등이다.
  * 그러나 수렴하지 않는 활성화함수(ReLu)등은 도움이 안된다.
  * 배치 정규화는 RNN에 효율적이지 않다.
    * 은닉이 아닌 입력에 적용할 때만 도움이 된다.
  * 잘맞는 정규화는 층 정규화이다.
    * 배치차원의 정규화가 아닌 특성 차원에 대해 정규화한다.
    * 샘플에 독립적으로 타임 스텝마다 동적으로 필요한 통계를 계산할 수 있다.
    * 이는 훈련과 테스트에서 동일한 방식으로 작동하는 것을 의미한다.
    * 훈련 세트의 모든 샘플에 대한 특성 통계를 추정하기 위해 지수 이동 평균이 필요하지 않다.
    * tf.keras를 이용한 메모리 셀 안에 층 정규화 구현

In [38]:
from tensorflow.keras.layers import LayerNormalization
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                          activation=None)
        self.layer_norm = LayerNormalization()
        self.activation = keras.activations.get(activation)
    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        if inputs is not None:
            batch_size = tf.shape(inputs)[0]
            dtype = inputs.dtype
        return [tf.zeros([batch_size, self.state_size], dtype=dtype)]
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [40]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                     input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=4,
                    validation_data=(X_valid, Y_valid))

Epoch 1/4
219/219 [==============================] - 13s 47ms/step - loss: 0.1400 - last_time_step_mse: 0.1432 - val_loss: 0.0490 - val_last_time_step_mse: 0.0485
Epoch 2/4
219/219 [==============================] - 10s 43ms/step - loss: 0.0317 - last_time_step_mse: 0.0284 - val_loss: 0.0228 - val_last_time_step_mse: 0.0198
Epoch 3/4
219/219 [==============================] - 9s 43ms/step - loss: 0.0190 - last_time_step_mse: 0.0145 - val_loss: 0.0169 - val_last_time_step_mse: 0.0118
Epoch 4/4
219/219 [==============================] - 10s 44ms/step - loss: 0.0156 - last_time_step_mse: 0.0106 - val_loss: 0.0145 - val_last_time_step_mse: 0.0081


### 단기 기억 문제 해결하기
* RNN은 정보가 몇 스탭후 사라진다.
* LSTM이 장기 메모리를 가진셀중 가장 인기 있다.
### LSTM 셀
* 장 단기 메모리는 1997년 소개 됬고 훈련이 빠르게 수렴하고 장기간의 의존성이 있다.
* 케라스에서는 SimpleRNN 대신 LSTM층을 사용하면 된다.
* GPU에 최적화된 구현을 사용하므로 일반적으로 선호된다.
* 장기기억 C(t-1)은 삭제게이트에서 기억을 잃고, 덧샘 연산으로 새로운 기억 이루를 추가한다. 입력 게이트에서 선택한 기억을 추가한다.
  * 장기 상태가 복사되어 tanh함수로 전달되고 출력 게이트에 의해서 걸러진다.
* 입력을 인식하고 장기상태에 저장하고 이를 보존하고 필요할때마다 추출하는 것을 학습.
---
### 핍홀 연결
* 제어기는 x와 이전 단기 상태 h만 바라볼 수 있다.
* 게이트 제어기에 장기 상태도 조금 노출시켜 좀 더 많은 문맥을 감지하게 만들면 좋을 수 있다.
* c가 f와i에 입력으로 추가된다.
---
# GRU셀
* 게이트 순환 유닛 2014 조경현 논문
* LSTM의 간소화된 버전이고 유사하게 보인다.


In [43]:
model=keras.models.Sequential([
                               keras.layers.LSTM(20,return_sequences=True,input_shape=[None,1]),
                               keras.layers.LSTM(20,return_sequences=True),
                               keras.layers.TimeDistributed(keras.layers.Dense(10))
])
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=4,
                    validation_data=(X_valid, Y_valid))

Epoch 1/4
219/219 [==============================] - 14s 48ms/step - loss: 0.0533 - last_time_step_mse: 0.0421 - val_loss: 0.0265 - val_last_time_step_mse: 0.0079
Epoch 2/4
219/219 [==============================] - 10s 44ms/step - loss: 0.0225 - last_time_step_mse: 0.0060 - val_loss: 0.0202 - val_last_time_step_mse: 0.0048
Epoch 3/4
219/219 [==============================] - 9s 43ms/step - loss: 0.0184 - last_time_step_mse: 0.0045 - val_loss: 0.0171 - val_last_time_step_mse: 0.0038
Epoch 4/4
219/219 [==============================] - 9s 43ms/step - loss: 0.0159 - last_time_step_mse: 0.0039 - val_loss: 0.0152 - val_last_time_step_mse: 0.0049


In [46]:
model.predict(X_test)[0][0]

array([ 0.11896153,  0.1545727 ,  0.16770215,  0.15585402,  0.10616609,
        0.1209249 ,  0.03158204,  0.01551342, -0.03752943, -0.06451225],
      dtype=float32)

In [47]:
model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=4,
                    validation_data=(X_valid, Y_valid))

Epoch 1/4
219/219 [==============================] - 48s 50ms/step - loss: 0.0514 - last_time_step_mse: 0.0528 - val_loss: 0.0274 - val_last_time_step_mse: 0.0234
Epoch 2/4
219/219 [==============================] - 10s 48ms/step - loss: 0.0246 - last_time_step_mse: 0.0200 - val_loss: 0.0231 - val_last_time_step_mse: 0.0175
Epoch 3/4
219/219 [==============================] - 10s 47ms/step - loss: 0.0213 - last_time_step_mse: 0.0156 - val_loss: 0.0190 - val_last_time_step_mse: 0.0137
Epoch 4/4
219/219 [==============================] - 10s 46ms/step - loss: 0.0165 - last_time_step_mse: 0.0097 - val_loss: 0.0145 - val_last_time_step_mse: 0.0078


### 1D 합성곱 층을 사용해 시퀀스 처리하기
* 1D합성곱 층으로 1D 특성 맵을 출력한다.
* 각 커널은 매우 짧은 하나의 순차패턴을 감지하도록 학습된다.
* 10개의 커널을 사용하면 이 층의 출력은 모두 길이가 같은 10개의 1차원 시퀀스로 구성된다.
* 이는 순환층과 섞어 신경망을 구성할 수 있다는 뜻이다.


In [48]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train[:, 3::2], epochs=5,
                    validation_data=(X_valid, Y_valid[:, 3::2]))

Epoch 1/5
219/219 [==============================] - 10s 29ms/step - loss: 0.0436 - last_time_step_mse: 0.0439 - val_loss: 0.0218 - val_last_time_step_mse: 0.0200
Epoch 2/5
219/219 [==============================] - 6s 25ms/step - loss: 0.0149 - last_time_step_mse: 0.0125 - val_loss: 0.0107 - val_last_time_step_mse: 0.0083
Epoch 3/5
219/219 [==============================] - 6s 25ms/step - loss: 0.0096 - last_time_step_mse: 0.0067 - val_loss: 0.0088 - val_last_time_step_mse: 0.0055
Epoch 4/5
219/219 [==============================] - 6s 25ms/step - loss: 0.0079 - last_time_step_mse: 0.0050 - val_loss: 0.0076 - val_last_time_step_mse: 0.0038
Epoch 5/5
219/219 [==============================] - 6s 25ms/step - loss: 0.0067 - last_time_step_mse: 0.0038 - val_loss: 0.0063 - val_last_time_step_mse: 0.0037


### WaveNet